In [89]:
# environment/LLM/smart_citation_llm_rater.py

from environment.LLM.rater import LLMRater

class SmartCitationLLMRater(LLMRater):
    def __init__(self, llm, current_items_features_list=[], previous_items_features_list=[], llm_render=False, llm_query_explanation=False):
        super().__init__(llm, current_items_features_list, previous_items_features_list, llm_render, llm_query_explanation)
        self.request_scale = "0-9"
        self.system_prompt = (
            '''You are simulating a user rating scientific papers.
                When rating, you MUST consider the following:

                - Topic Matching: Rate higher if paper topics match user's interests.
                - Novelty Preference: The user has a novelty preference score from 0 (does not care about recency) to 1 (cares strongly about recency). 
                You should reward papers with a Normalized Year closer to 1 based on the user's novelty preference.
                - Reputability Bias: The user has a reputability bias score from 0 (does not care about citations) to 1 (cares strongly about highly cited papers).
                You should reward papers with a Normalized Citations (Reputability) closer to 1 based on the user's reputability bias.
                
                You must **combine these factors** logically to decide if the user would rate the paper highly or not.
                ONLY output a single rating from 0 (not interested) to 9 (extremely interested).
'''
        )

        # Interested in: Web visibility and informetrics. Prefers novelty: 0.74, reputability bias: 0.47
    def _get_few_shot_prompts(self):
        # (Optional for smarter few-shot prompts later)
        return []
    def _get_prompt(self, user, item, num_interacted, interactions, retrieved_items):
        user_info = (
            f"User Profile:\n"
            f" {user.description}"
            # f"- Novelty Preference (0-1): {user.novelty_preference:.2f}\n"
            # f"- Reputability Bias (0-1): {user.reputability_bias:.2f}\n"
        )

        paper_info = (
            f"Paper Details:\n"
            f"- Title: {item.title}\n"
            f"- Topics: {', '.join(item.topics)}\n"
            f"- Normalized Year (0-1): {item.norm_year:.2f}\n"
            f"- Normalized Citations (Reputability) (0-1): {item.norm_cite:.2f}\n"
        )
        # print(f"User Info: {user_info}")
        # print(f"Paper Info: {paper_info}")
        question = "Question:\nHow much would the user like this paper? (ONLY output a number from 0 to 9)"

        full_prompt = f"{self.system_prompt}\n\n{user_info}\n\n{paper_info}\n\n{question}"

        return [{"role": "user", "content": full_prompt}]

    def adjust_rating_in(self, rating):
        return rating

    def adjust_rating_out(self, rating):
        return rating

    def adjust_text_in(self, text):
        return text


# environment/LLM/small_hf_llm.py

from environment.LLM.llm import LLM
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

class SmallHuggingfaceLLM(LLM):
    def __init__(self, model_name="google/flan-t5-large", device="cpu"):
        super().__init__(model_name)
        self.device = device
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
        self.model.eval()

    def _generate(self, prompt):
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        outputs = self.model.generate(
            **inputs,
            max_new_tokens=10,
            pad_token_id=self.tokenizer.eos_token_id
        )
        text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return text

    def request_rating_0_9(self, system_prompt, dialog):
        prompt = self._dialog_to_text(dialog)
        response = self._generate(prompt)
        return prompt, self._extract_number(response, 0, 9)

    def request_rating_1_10(self, system_prompt, dialog):
        prompt = self._dialog_to_text(dialog)
        response = self._generate(prompt)
        return prompt, self._extract_number(response, 1, 10)

    def request_rating_text(self, system_prompt, dialog):
        prompt = self._dialog_to_text(dialog)
        response = self._generate(prompt)
        return prompt, response

    def request_explanation(self, system_prompt, dialog):
        prompt = self._dialog_to_text(dialog)
        response = self._generate(prompt)
        return prompt, response

    def _dialog_to_text(self, dialog):
        return "\n".join([f"{d['role'].capitalize()}: {d['content']}" for d in dialog])

    def _extract_number(self, text, min_val, max_val):
        import re
        numbers = re.findall(r'\d+', text)
        for num in numbers:
            n = int(num)
            if min_val <= n <= max_val:
                return str(n)
        return str((min_val + max_val) // 2)  # fallback: middle value



llm = SmallHuggingfaceLLM(model_name="google/flan-t5-small", device="cpu")
rater = SmartCitationLLMRater(llm)

In [90]:
import sys
import os
sys.path.append(os.path.abspath(".."))  

from stable_baselines3 import DQN
from environment.env import Simulatio4RecSys
from environment.citations.citation_loader import CitationsLoader
from environment.users.citation_users_loader import CitationUsersLoader
from environment.citations.citations_retrieval import CitationsRetrieval
from environment.items_selection import GreedySelector
from environment.reward_shaping import IdentityRewardShaping
from environment.reward_perturbator import NoPerturbator
from environment.flat_obs_wrapper import FlatObsWrapper
from environment.LLM.rater import DummyLLMRater
from environment.LLM.dummy_llm import DummyLLM
from stable_baselines3.common.monitor import Monitor


csv_path = "../environment/citations/datasets/cleaned-scientometrics-and-bibliometrics-research.csv"
users_path = "../environment/users/datasets/citation_users.json"

# Load
items_loader = CitationsLoader(csv_path)
users_loader = CitationUsersLoader(users_path)

raw_env = Simulatio4RecSys(
    render_mode="human",
    items_loader=items_loader,
    users_loader=users_loader,
    items_selector=GreedySelector(),
    reward_perturbator=NoPerturbator(),
    items_retrieval=CitationsRetrieval(),
    reward_shaping=IdentityRewardShaping(),
    llm_rater=rater
)

env = FlatObsWrapper(raw_env)
env = Monitor(env)

# Load model
model = DQN.load("models/dqn_citation_recommender_llm", env=env, device="cpu")


Wrapping the env in a DummyVecEnv.


In [91]:
import numpy as np

def evaluate_precision_at_k(env, model, rater, users_loader, K=5, threshold=6):
    """
    Evaluate Precision@K across all users.

    Args:
        env: The environment (wrapped with Monitor, FlatObsWrapper, etc.)
        model: Trained RL model (e.g., DQN, A2C)
        rater: LLMRater or RuleBasedRater to simulate user feedback
        users_loader: Loader to access users
        K: Number of top recommendations
        threshold: Minimum score to count as a "click"

    Returns:
        mean_precision: Mean precision@K across users
    """

    all_precisions = []
    n_users = len(users_loader.get_users())

    for user_idx in range(n_users)[:2]:
        # Reset user
        obs = env.reset()[0]
        user = users_loader.get_users()[user_idx]

        clicked = []

        for _ in range(K):
            action, _ = model.predict(obs, deterministic=True)
            action = int(action)
            item_id = env.unwrapped.action_to_item[action]
            paper = env.unwrapped.items_loader.load_items_from_ids([item_id])[0]
            print(f"User {user_idx}: Paper ID: {item_id}, Title: {paper.title}")

            # Get simulated "rating" from user for this paper
            rating, _, _ = rater.query(user, paper, 0, [], [])

            # Check if user "clicked" (liked it)
            click = (rating >= threshold)
            clicked.append(click)

            # Step environment
            obs, reward, done, truncated, info = env.step(action)

            if done or truncated:
                break

        precision = np.sum(clicked) / K
        all_precisions.append(precision)

    mean_precision = np.mean(all_precisions)
    return mean_precision


In [92]:
mean_precision = evaluate_precision_at_k(env, model, rater, users_loader, K=5, threshold=1)
print(f"📊 Precision@5 = {mean_precision:.4f}")


User 0: Paper ID: 825, Title: Is commercialization good or bad for science? Individual-level evidence from the Max Planck Society
User 0: Paper ID: 825, Title: Is commercialization good or bad for science? Individual-level evidence from the Max Planck Society
User 0: Paper ID: 825, Title: Is commercialization good or bad for science? Individual-level evidence from the Max Planck Society
User 0: Paper ID: 825, Title: Is commercialization good or bad for science? Individual-level evidence from the Max Planck Society
User 0: Paper ID: 825, Title: Is commercialization good or bad for science? Individual-level evidence from the Max Planck Society
User 1: Paper ID: 825, Title: Is commercialization good or bad for science? Individual-level evidence from the Max Planck Society
User 1: Paper ID: 825, Title: Is commercialization good or bad for science? Individual-level evidence from the Max Planck Society
User 1: Paper ID: 825, Title: Is commercialization good or bad for science? Individual-lev

In [94]:
import numpy as np

def evaluate_precision_at_k(env, model, rater, users_loader, K=5, threshold=6):
    """
    Evaluate Precision@K across first `num_users_eval` users.

    Args:
        env: The environment (wrapped with Monitor, FlatObsWrapper, etc.)
        model: Trained RL model (e.g., DQN, A2C)
        rater: LLMRater or RuleBasedRater to simulate user feedback
        users_loader: Loader to access users
        K: Number of top recommendations
        threshold: Minimum score to count as a "click"
        num_users_eval: Number of users to evaluate (default 2)

    Returns:
        mean_precision: Mean precision@K across users
    """

    all_precisions = []
    n_users = len(users_loader.get_users())
    # num_users_eval = min(num_users_eval, n_users)

    for user_idx in range(n_users):
        obs = env.reset()[0]
        user = users_loader.get_users()[user_idx]

        clicked = []
        recommended_actions = set()

        print(f"\n🔵 User {user_idx}: {user.description}")

        for rec_idx in range(K):
            action, _ = model.predict(obs, deterministic=True)
            action = int(action)

            # Avoid recommending the same paper again
            while action in recommended_actions:
                action = env.action_space.sample()
                action = int(action)

            recommended_actions.add(action)

            item_id = env.unwrapped.action_to_item[action]
            paper = env.unwrapped.items_loader.load_items_from_ids([item_id])[0]

            print(f"    ➔ Recommendation {rec_idx+1}: Paper ID {item_id} - {paper.title} - {paper.topics} - {paper.norm_year:.2f} - {paper.norm_cite:.2f}")

            # Get simulated "rating" from user for this paper
            rating, _, _ = rater.query(user, paper, 0, [], [])
            # print(rating)

            # Check if user "clicked" (liked it)
            click = (rating >= threshold)
            clicked.append(click)
            print(click)

            # Step environment
            obs, reward, done, truncated, info = env.step(action)

            if done or truncated:
                obs = env.reset()[0]

        precision = np.sum(clicked) / K
        all_precisions.append(precision)

    mean_precision = np.mean(all_precisions)
    print(f"\n📊 Precision@{K}: {mean_precision:.4f}")
    return mean_precision
mean_precision = evaluate_precision_at_k(env, model, rater, users_loader, K=6, threshold=4)


🔵 User 0: Interested in: Web visibility and informetrics. Prefers novelty: 0.74, reputability bias: 0.47
    ➔ Recommendation 1: Paper ID 825 - Is commercialization good or bad for science? Individual-level evidence from the Max Planck Society - ['scientometrics and bibliometrics research', 'Innovation and Knowledge Management', 'Innovation Policy and R&D'] - 0.98 - 0.54
True
    ➔ Recommendation 2: Paper ID 24496 - Indicateurs de notoriété des revues scientifiques, des chercheurs et des articles publiés en santé : perspective pharmaceutique - ['scientometrics and bibliometrics research', 'Academic Publishing and Open Access', 'Meta-analysis and systematic reviews'] - 0.99 - 0.09
False
    ➔ Recommendation 3: Paper ID 24250 - Value and Quality Measures for Chemistry Research Journals - ['scientometrics and bibliometrics research'] - 0.98 - 0.11
False
    ➔ Recommendation 4: Paper ID 38696 - Open access policy at Reviews in Health Care - ['scientometrics and bibliometrics research', 'H

In [67]:
# environment/LLM/rule_based_citation_rater.py


import re
from sentence_transformers import SentenceTransformer, util
from abc import ABC, abstractmethod
import numpy as np

class Rater(ABC):
    def __init__(self, llm=None, current_items_features_list=None, previous_items_features_list=None, llm_render=False, llm_query_explanation=False):
        self.llm = llm
        self.llm_render = llm_render
        self.llm_query_explanation = llm_query_explanation
        self.current_items_features_list = current_items_features_list if current_items_features_list else []
        self.previous_items_features_list = previous_items_features_list if previous_items_features_list else []

    @abstractmethod
    def query(self, user, item, num_interacted, interactions, retrieved_items):
        pass

class RuleBasedCitationRater(Rater):
    def __init__(self, 
                 bert_model=None, 
                 weight_topic=0.7, 
                 weight_novelty=0.2, 
                 weight_reputability=0.1):
        """
        Rule-Based Citation Rater.
        """
        super().__init__(llm=None)
        
        if bert_model is None:
            self.bert_model = SentenceTransformer('all-MiniLM-L6-v2')  # default lightweight model
        else:
            self.bert_model = bert_model

        self.weight_topic = weight_topic
        self.weight_novelty = weight_novelty
        self.weight_reputability = weight_reputability

    def parse_user_preferences(self, user):
        """
        Parse novelty and reputability preferences from user description.
        """
        description = user.description

        novelty_match = re.search(r'Prefers novelty: ([0-9.]+)', description)
        reputability_match = re.search(r'reputability bias: ([0-9.]+)', description)
        
        

        novelty = float(novelty_match.group(1)) if novelty_match else 0.5
        reputability = float(reputability_match.group(1)) if reputability_match else 0.5

        return novelty, reputability

    def compute_semantic_topic_score(self, user, item):
        """
        Compute cosine similarity between user interests and paper (title + topics).
        """
        user_text = user.description.split("Prefers novelty")[0].lower()
        paper_text = (item.title + " " + " ".join(item.topics)).lower()


        # Encode
        user_emb = self.bert_model.encode(user_text, convert_to_tensor=True)
        paper_emb = self.bert_model.encode(paper_text, convert_to_tensor=True)

        # Cosine similarity
        similarity = util.cos_sim(user_emb, paper_emb).item()
        similarity = max(0, similarity)  # Clamp to [0,1]


        return similarity

    def compute_novelty_score(self, novelty_pref, item):
        """
        Higher score if normalized year matches user's novelty preference.
        """
        return 1.0 - abs(novelty_pref - item.norm_year)

    def compute_reputability_score(self, reputability_bias, item):
        """
        Higher score if normalized citations match user's reputability bias.
        """
        return 1.0 - abs(reputability_bias - item.norm_cite)

    def query(self, user, item, num_interacted=None, interactions=None, retrieved_items=None):
        """
        Query rating function — must match (rating, explanation, prompt).
        """
        novelty_pref, reputability_bias = self.parse_user_preferences(user)

        topic_score = self.compute_semantic_topic_score(user, item)
        novelty_score = self.compute_novelty_score(novelty_pref, item)
        reputability_score = self.compute_reputability_score(reputability_bias, item)
        # print(f"Topic score: {topic_score}, Novelty score: {novelty_score}, Reputability score: {reputability_score}")
        final_score = (
            self.weight_topic * topic_score +
            self.weight_novelty * novelty_score +
            self.weight_reputability * reputability_score
        )

        rating = round(final_score * 9)
        rating = min(9, max(0, rating))  # Clamp
        # print(f"Rating: {rating}")
        # print(np.random.rand())
        # clicked = np.random.rand() < rating

        # We return rating, empty explanation, empty prompt (for compatibility)
        return int(rating), "", ""



from sentence_transformers import SentenceTransformer, util

bert_model = SentenceTransformer('all-MiniLM-L6-v2')  # lightweight and fast
rater = RuleBasedCitationRater(bert_model=bert_model)


csv_path = "../environment/citations/datasets/cleaned-scientometrics-and-bibliometrics-research.csv"
users_path = "../environment/users/datasets/citation_users.json"

items_loader = CitationsLoader(csv_path)
users_loader = CitationUsersLoader(users_path)

# --- Environment ---
raw_env = Simulatio4RecSys(
    render_mode="human",
    items_loader=items_loader,
    users_loader=users_loader,
    items_selector=GreedySelector(),
    reward_perturbator=NoPerturbator(),
    items_retrieval=CitationsRetrieval(),
    reward_shaping=IdentityRewardShaping(),
    llm_rater=rater    
)

env = FlatObsWrapper(raw_env)
env = Monitor(env)
model = DQN.load("models/dqn_citation_recommender_rule", env=env, device="cpu")

Wrapping the env in a DummyVecEnv.


In [ ]:
import numpy as np

def evaluate_precision_at_k(env, model, rater, users_loader, K=5, threshold=6):
    """
    Evaluate Precision@K across first `num_users_eval` users.

    Args:
        env: The environment (wrapped with Monitor, FlatObsWrapper, etc.)
        model: Trained RL model (e.g., DQN, A2C)
        rater: LLMRater or RuleBasedRater to simulate user feedback
        users_loader: Loader to access users
        K: Number of top recommendations
        threshold: Minimum score to count as a "click"
        num_users_eval: Number of users to evaluate (default 2)

    Returns:
        mean_precision: Mean precision@K across users
    """

    all_precisions = []
    n_users = len(users_loader.get_users())
    # num_users_eval = min(num_users_eval, n_users)

    for user_idx in range(n_users):
        obs = env.reset()[0]
        user = users_loader.get_users()[user_idx]

        clicked = []
        recommended_actions = set()

        print(f"\n🔵 User {user_idx}: {user.description}")

        for rec_idx in range(K):
            action, _ = model.predict(obs, deterministic=True)
            action = int(action)

            # Avoid recommending the same paper again
            while action in recommended_actions:
                action = env.action_space.sample()
                action = int(action)

            recommended_actions.add(action)

            item_id = env.unwrapped.action_to_item[action]
            paper = env.unwrapped.items_loader.load_items_from_ids([item_id])[0]

            print(f"    ➔ Recommendation {rec_idx+1}: Paper ID {item_id} - {paper.title} - {paper.topics}")

            # Get simulated "rating" from user for this paper
            rating, _, _ = rater.query(user, paper, 0, [], [])
            # print(rating)

            # Check if user "clicked" (liked it)
            click = rating >= threshold
            clicked.append(click)
            print(click)

            # Step environment
            obs, reward, done, truncated, info = env.step(action)

            if done or truncated:
                obs = env.reset()[0]

        precision = np.sum(clicked) / K
        all_precisions.append(precision)

    mean_precision = np.mean(all_precisions)
    print(f"\n📊 Precision@{K}: {mean_precision:.4f}")
    return mean_precision
mean_precision = evaluate_precision_at_k(env, model, rater, users_loader, K=5, threshold=3.5)


🔵 User 0: Interested in: Web visibility and informetrics. Prefers novelty: 0.74, reputability bias: 0.47
    ➔ Recommendation 1: Paper ID 27122 - The Problem of Measurement and Evaluation of Individual Scientific Productivity - ['scientometrics and bibliometrics research', 'Academic Publishing and Open Access']
True
    ➔ Recommendation 2: Paper ID 6434 - Do You Sincerely Want to Be Cited? Or: Read Before You Cite - ['scientometrics and bibliometrics research', 'Complex Network Analysis Techniques']
True
    ➔ Recommendation 3: Paper ID 18362 - Open access in addictions, associated data and countries collaboration - ['scientometrics and bibliometrics research']
False
    ➔ Recommendation 4: Paper ID 29522 - Merging Departments: A Unique Opportunity for Research - ['scientometrics and bibliometrics research', 'Health and Medical Research Impacts']
False
    ➔ Recommendation 5: Paper ID 25492 - Commentary: Peer Review in Scientific Publication - ['scientometrics and bibliometrics resear